In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('new_data_full.csv')
df = df.loc[:, ~df.columns.str.contains('9')]
df = df.drop(columns=['Division','Year']).dropna()
df['Soil Type'] = df['Soil Type'].astype('category')
df['Soil Type'] = df['Soil Type'].cat.codes

In [9]:
df

,1_Average of Soil Temp. 005 cm,1_Average of Soil Temp. 020 cm,1_Average of Soil Temp. 050 cm,1_Average of Soil Temp. 100 cm,1_Average of Soil Moisture 005 cm,1_Average of Soil Moisture 020 cm,1_Average of Soil Moisture 050 cm,1_Average of Soil Moisture 100 cm,1_Average of Long Precipitation,1_Average of Precip.,...,8_Average of Soil Moisture 020 cm,8_Average of Soil Moisture 050 cm,8_Average of Soil Moisture 100 cm,8_Average of Long Precipitation,8_Average of Precip.,8_Average of Max Temperature,8_Average of Average Temperature,8_Average of Min Temperature,Soil Type,Yield
4,-5.20,-4.40,-2.70,-0.10,11.00,13.70,16.50,18.30,1291.30,18.20,...,18.5,23.90,22.40,1534.20,45.85,26.80,18.65,10.50,1,659.30
5,-2.90,-2.40,-0.90,1.20,17.90,16.80,19.70,19.30,1619.60,13.55,...,21.2,23.20,25.30,197.05,56.90,26.80,18.65,10.50,1,605.01
6,-2.95,-2.25,-1.05,1.20,17.25,15.55,16.80,20.05,282.45,34.75,...,20.1,20.70,24.40,525.35,25.70,26.80,18.65,10.50,1,582.47
7,-4.15,-3.05,-0.90,1.50,15.80,14.75,15.05,20.25,610.75,4.75,...,18.4,18.90,23.65,854.00,17.85,26.80,18.65,10.50,1,582.90
8,-6.70,-5.40,-2.90,0.00,16.50,13.00,17.10,19.00,939.40,9.85,...,16.9,21.10,22.30,1182.30,41.30,26.80,18.65,10.50,1,612.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,-2.00,-1.25,-0.25,1.05,13.87,19.20,24.47,26.43,1987.37,17.43,...,21.8,25.93,26.97,209.97,40.07,21.67,15.00,8.37,3,340.03
138,-4.10,-3.40,-2.50,-0.90,8.40,9.60,22.00,21.40,327.00,30.20,...,19.8,34.40,33.70,579.80,26.30,21.30,14.50,7.70,2,632.57
139,-5.20,-3.70,-2.00,-0.10,8.40,10.50,24.80,30.20,703.70,27.20,...,12.0,27.50,31.30,957.00,39.10,21.30,14.50,7.70,2,576.20
140,-1.60,-1.10,-0.50,0.80,12.80,12.40,24.50,27.30,1080.90,32.40,...,16.7,28.20,29.60,1333.70,27.80,21.30,14.50,7.70,2,584.40


In [11]:
X = df.iloc[:, :-1]
y = df['Yield']
col_list = X.columns
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
pd.DataFrame(X,columns=col_list)

,1_Average of Soil Temp. 005 cm,1_Average of Soil Temp. 020 cm,1_Average of Soil Temp. 050 cm,1_Average of Soil Temp. 100 cm,1_Average of Soil Moisture 005 cm,1_Average of Soil Moisture 020 cm,1_Average of Soil Moisture 050 cm,1_Average of Soil Moisture 100 cm,1_Average of Long Precipitation,1_Average of Precip.,...,8_Average of Soil Moisture 005 cm,8_Average of Soil Moisture 020 cm,8_Average of Soil Moisture 050 cm,8_Average of Soil Moisture 100 cm,8_Average of Long Precipitation,8_Average of Precip.,8_Average of Max Temperature,8_Average of Average Temperature,8_Average of Min Temperature,Soil Type
0,-1.048368,-1.167978,-1.132197,-0.647646,-0.895264,-0.853176,-1.179859,-1.781559,-0.009966,0.447406,...,0.582305,-0.997461,-0.884736,-1.856252,0.654691,-0.031920,2.219414,2.353952,2.361608,-0.131024
1,0.220963,0.009464,0.109787,0.373906,0.608567,-0.376768,-0.713180,-1.624893,0.540405,-0.041628,...,0.518023,-0.586540,-1.002641,-1.308121,-1.365940,0.473059,2.219414,2.353952,2.361608,-0.131024
2,0.193369,0.097772,0.006289,0.373906,0.466902,-0.568868,-1.136108,-1.507392,-1.701230,2.187946,...,-0.400292,-0.753952,-1.423731,-1.478230,-0.869830,-0.952764,2.219414,2.353952,2.361608,-0.131024
3,-0.468891,-0.373204,0.109787,0.609648,0.150880,-0.691812,-1.391323,-1.476059,-1.150859,-0.967112,...,-0.556406,-1.012680,-1.726916,-1.619988,-0.373192,-1.311505,2.219414,2.353952,2.361608,-0.131024
4,-1.876192,-1.756699,-1.270195,-0.569065,0.303442,-0.960751,-1.092357,-1.671893,-0.599901,-0.430752,...,0.361909,-1.240969,-1.356357,-1.875153,0.122918,-0.239853,2.219414,2.353952,2.361608,-0.131024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.717657,0.686493,0.558281,0.256034,-0.269757,-0.007937,-0.017537,-0.507858,1.156945,0.366426,...,-0.697827,-0.495224,-0.542811,-0.992473,-1.346416,-0.296063,-0.897549,-0.811030,-0.416939,1.594130
75,-0.441297,-0.579257,-0.994198,-1.276293,-1.461925,-1.483263,-0.377755,-1.295892,-1.626545,1.709429,...,0.361909,-0.799610,0.883841,0.279568,-0.787548,-0.925344,-1.122359,-1.244589,-1.290942,0.731553
76,-1.048368,-0.755873,-0.649203,-0.647646,-1.461925,-1.344951,0.030589,0.082776,-0.995035,1.393923,...,-1.217593,-1.986715,-0.278367,-0.174057,-0.217543,-0.340391,-1.122359,-1.244589,-1.290942,0.731553
77,0.938410,0.774801,0.385784,0.059582,-0.502960,-1.052959,-0.013162,-0.371558,-0.362686,1.940800,...,-0.409476,-1.271408,-0.160462,-0.495375,0.351706,-0.856795,-1.122359,-1.244589,-1.290942,0.731553


In [13]:
def baseline_ensemble(X,y, lr_model_name='Linear Regression', rf_model_name='Random Forest Regressor', rf_n_estimators=100, rf_max_depth=None, lasso_model_name='Lasso Regression', lasso_alpha=1.0, elasticnet_model_name='Elastic Net Regression', elasticnet_alpha=1.0, elasticnet_l1_ratio=0.5, svr_model_name='Support Vector Regression', svr_kernel='rbf', svr_C=1.0, svr_epsilon=0.1):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True)
    
    # Initialize a Linear Regression object
    lr_model = LinearRegression()
    
    # Fit the Linear Regression model to the training data
    lr_model.fit(X_train, y_train)
    
    # Calculate the training and testing scores for the Linear Regression model
    lr_score_train = lr_model.score(X_train, y_train)
    lr_score_test = lr_model.score(X_test, y_test)
    lr_mse_train = mean_squared_error(y_train, lr_model.predict(X_train),squared=False)
    lr_mse_test = mean_squared_error(y_test, lr_model.predict(X_test),squared=False)
    
    # Initialize a Random Forest Regressor object
    rf_model = RandomForestRegressor(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=42)
    
    # Fit the Random Forest Regressor model to the training data
    rf_model.fit(X_train, y_train)
    
    # Calculate the training and testing scores for the Random Forest Regressor model
    rf_score_train = rf_model.score(X_train, y_train)
    rf_score_test = rf_model.score(X_test, y_test)
    rf_mse_train = mean_squared_error(y_train, rf_model.predict(X_train),squared=False)
    rf_mse_test = mean_squared_error(y_test, rf_model.predict(X_test),squared=False)
    
    # Initialize a Lasso Regression object
    lasso_model = Lasso(alpha=lasso_alpha, random_state=42)
    
    # Fit the Lasso Regression model to the training data
    lasso_model.fit(X_train, y_train)
    
    # Calculate the training and testing scores for the Lasso Regression model
    lasso_score_train = lasso_model.score(X_train, y_train)
    lasso_score_test = lasso_model.score(X_test, y_test)
    lasso_mse_train = mean_squared_error(y_train, lasso_model.predict(X_train),squared=False)
    lasso_mse_test = mean_squared_error(y_test, lasso_model.predict(X_test),squared=False)
    
    # Initialize an Elastic Net Regression object
    elasticnet_model = ElasticNet(alpha=elasticnet_alpha, l1_ratio=elasticnet_l1_ratio, random_state=42)
    
    # Fit the Elastic Net Regression model to the training data
    elasticnet_model.fit(X_train, y_train)
    
    # Calculate the training and testing scores for the Elastic Net Regression model
    elasticnet_score_train = elasticnet_model.score(X_train, y_train)
    elasticnet_score_test = elasticnet_model.score(X_test, y_test)
    elasticnet_mse_train = mean_squared_error(y_train, elasticnet_model.predict(X_train),squared=False)
    elasticnet_mse_test = mean_squared_error(y_test, elasticnet_model.predict(X_test),squared=False)
    
    # Initialize a Support Vector Regression object
    svr_model = SVR(kernel=svr_kernel, C=svr_C, epsilon=svr_epsilon)
    
    # Fit the Support Vector Regression model to the training data
    svr_model.fit(X_train, y_train)
    
    # Calculate the training and testing scores for the Support Vector Regression model
    svr_score_train = svr_model.score(X_train, y_train)
    svr_score_test = svr_model.score(X_test, y_test)
    svr_mse_train = mean_squared_error(y_train, svr_model.predict(X_train),squared=False)
    svr_mse_test = mean_squared_error(y_test, svr_model.predict(X_test),squared=False)
    
    # Create a dataframe with the scores
    score_df = pd.DataFrame(({'Model': [lr_model_name,rf_model_name,lasso_model_name,elasticnet_model_name,svr_model_name],
                             'R2 Training Score': [lr_score_train,rf_score_train,lasso_score_train,elasticnet_score_train,svr_score_train],
                            'MSE Training Score': [lr_mse_train,rf_mse_train,lasso_mse_train,elasticnet_mse_train,svr_mse_train],
                             'R2 Testing Score': [lr_score_test,rf_score_test,lasso_score_test,elasticnet_score_test,svr_score_test],
                             'MSE Testing Score': [lr_mse_test,rf_mse_test,lasso_mse_test,elasticnet_mse_test,svr_mse_test]}))
    return score_df

In [14]:
baseline_ensemble(X,y)

/Users/ronan/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1642.8208137655747, tolerance: 414.0857316092308
  model = cd_fast.enet_coordinate_descent(


,Model,R2 Training Score,MSE Training Score,R2 Testing Score,MSE Testing Score
0,Linear Regression,1.000000,8.622263e-13,0.847278,121.319582
1,Random Forest Regressor,0.978823,4.106526e+01,0.893361,101.376467
2,Lasso Regression,0.988163,3.070162e+01,0.923273,85.991119
3,Elastic Net Regression,0.797191,1.270830e+02,0.669240,178.540232
4,Support Vector Regression,-0.102907,2.963555e+02,-0.073077,321.584674


In [15]:
baseline_ensemble(X,y).to_csv('baseline_models_soil.csv',index=False)

/Users/ronan/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1642.8208137655747, tolerance: 414.0857316092308
  model = cd_fast.enet_coordinate_descent(
